In [1]:
# Instalando ibliotecas necessárias para as manipulaçãos

import numpy as np #p/ manipulação de matrizes
from skimage import io, data #manipulação e funções de manipulação de dados (dados foi usado apenas para teste)
import matplotlib.pyplot as plt #p/ demonstração de gráfico e das imagens
import os #acesso aos arquivos
import cv2 #manipulação do vídeo
import logging 

In [2]:
#formatando log para controle do fluxo
caminho = '../log/notebook.log'
logging.basicConfig(level=logging.INFO, filename=caminho, filemode='a', format="%(asctime)s - %(levelname)s - %(message)s", encoding='utf-8')

In [3]:
# Trecho para determinar a necessidade da equalização

def _desvio_padrao(frame_h:np.histogram)->tuple:
  media_ = np.mean(frame_h)
  logging.info("A média: %2.f", media_)
  dif_quadradas = (frame_h - media_) ** 2
  #logging.info("Matriz de diferenças quadradas: $s", str(dif_quadradas))
  media_diff_ = np.mean(dif_quadradas)
  logging.info("Media da Matriz de diferenças: %2.f", media_diff_)
  desvio_padrao = np.sqrt(media_diff_)
  return desvio_padrao, media_

def avalia_pic(matriz_imagem:np.array)->tuple:
    histograma = np.histogram(matriz_imagem, bins=256, range=(0,255))
    #plt.plot (histograma[0],'-k')
    desvio_, media_ = _desvio_padrao(histograma[0])
    logging.info("Desvio Padrão: %2.f", desvio_)
    coff = desvio_ / media_
    logging.info("Coeficiente de variação: %2.f", coff)
    if coff < 1.0: # a verificar
      y_n = False
    else: 
      y_n = True
    return y_n, histograma

In [4]:
# Funções para a equalização
def _funcao_transferencia(frame, frame_h:np.histogram):
  l,c = frame.shape 
  n = l*c 
  pr = frame_h[0]/ n
  return np.cumsum (pr)


def normalizacao_histogramica(histograma_:np.histogram, _transferencia, img:np.array):
  imgh = np.zeros_like(img).astype('float')
  l,c = img.shape
  for i in range(l):
    for j in range(c):
      imgh[i,j] = _transferencia[[img[i, j]]]
  return imgh

def demonstracao(img):
  _,ax = plt.subplots (1,2)
  ax[0].imshow ((img*255).astype('uint8'),cmap='gray',vmin=0,vmax=255)
  h = np.histogram (img,bins=256)
  ax[1].plot (h[0],'-k')
  logging.info("np.sum equalizado: %d", np.sum(h[0]))

In [5]:
#Capturar os videos
# Inicializar o objeto VideoCapture
caminho_video1 = '../tests/origem/VID_20240405_234240.mp4'
caminho_video2 = '../tests/origem/VID_20240405_234310.mp4'
caminho_video3 = '../tests/origem/VID-20240406-WA0059.mp4'

cap = cv2.VideoCapture(caminho_video1)
 
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        logging.error("Can't receive frame (stream end?). Exiting ...")
        break
    #logging.info(frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #plt.imshow(gray, cmap='gray')
    #plt.show()
    yeah, h_ = avalia_pic(gray)
    
    video_writer = cv2.VideoWriter(
        "../saida/video.avi",
        cv2.VideoWriter_fourcc(*"XVID"),
        30,
        (gray.shape[1], gray.shape[0]),
    )

    if yeah == True:
        func = _funcao_transferencia(gray, h_)
        imgh = normalizacao_histogramica(h_, func, gray)
        #demonstracao(imgh)
        video_writer.write(imgh)
    else:
        video_writer.write(gray)
        #_, ax = plt.subplots (1, 2)
        #ax[0].imshow(gray, cmap='gray')
        #ax[1].plot(h_[0], '-k')
        logging.info("np.sum não equalizado: %d", np.sum(h_[0]))
    
    
    video_writer.release()
 
cap.release()
cv2.destroyAllWindows()

C:\Users\Joana\AppData\Local\Temp\ipykernel_15944\1252220516.py:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  _,ax = plt.subplots (1,2)
